In [92]:
import urllib2 as url #url query
import ast #json to dict
import pandas as pd #convert listed dict to DataFrame
import numpy as np
import re
import inflect
import math


In [129]:
http_api = "http://www.omdbapi.com/"

#build query for exact name match
def omdbNameQuery(title,year=None):
    if year is None:
        qstr = http_api+"?t="
        qstr += str(title).replace(" ","%20")
    else:
        qstr += "&y="
        qstr += str(year)    
    
    qstr += "&tomatoes=true"
    return qstr


#build query for keyword search
def omdbSrchQuery(keyword,year=None):
    if year is None:
        qstr = http_api+"?s="
        qstr += str(keyword).replace(" ","%20")
    else:
        qstr += "&y="
        qstr += str(year)    
    
    #qstr += "&tomates=true" #not an option for search?
    return qstr

def getInts(str1):
    if(type(str1) == type("")):
        return [int(s) for s in re.split("\(|,|\)| |\[|\]|_|\.",str1) if s.isdigit()]
        #return [int(s) for s in str1.split() if s.isdigit()]
    else:
        return [""]
    
#array of bools for binary representation
def npbool(i):
    if i == 0:
        return [False]
    else :
        ba = np.empty(int(math.log(i,2)+1),dtype=bool)
        #i-=1
        for k in range(ba.shape[0]):
            ba[k] = bool(i>>k&1)

        return ba

In [93]:
#search for exact match
nam = "12 monkeys"
nums = getInts(nam)

for i in range(2**(len(nums))):
    print(i)
try:
    ret1 = url.urlopen(omdbNameQuery(nam)).read()
    #print(ret1)
    ret1d = ast.literal_eval(ret1) #convert to dict

    if(ast.literal_eval(ret1d['Response'])): #check if movie was present
        print("Title: "+ret1d['Title'])
        print("Year: "+ret1d['Year'])
        print("Type: "+ret1d['Type'])
        #print(ret1d.keys())
        print("Genre(s): "+ret1d['Genre'])
        print("IMDB Score: "+ret1d['imdbRating'])
        print("Rotten Tomato Metrics:")
        print(ret1d['tomatoUserRating'])
        print(ret1d['tomatoFresh'])
        print(ret1d['tomatoUserMeter'])
        print(ret1d['tomatoRating'])
        print(ret1d['tomatoURL'])
    else:
        print(ret1d['Error'])
    #print(ret1d)
except (url.URLError,NameError) as e:
    print("------------Error----------")
    print(e)
    

0
1
Title: 12 Monkeys
Year: 2015–
Type: series
Genre(s): Adventure, Drama, Mystery
IMDB Score: 7.6
Rotten Tomato Metrics:
N/A
N/A
N/A
N/A
N/A


In [158]:
nam = "1, 2 (3)"
nums = getInts(nam)
p = inflect.engine()

#since it's exponentiation and not multiplication, for nested
# for loops you would need a nest per entry...
a=0

for i in range(0,2**(len(nums))):
    #convert i to binary, the numerals are elements in a boolean array
    #if ith element =True, replace that number with words
    nstr = nam
    ba = npbool(i)
    #print(ba)
    for j in range(len(ba)):
        if ba[j] == True :
            nstr = nstr.replace(str(nums[j]),p.number_to_words(nums[j]))

    print(nstr)

print("-------------")
        
for j in range(0,2**(len(nums))):
    #convert i to binary, the numerals are elements in a boolean array
    #if ith element =True, replace that number with words
    nstr = nam
    for k in [i for i in npbool(j) if i == True] :
        nstr = nstr.replace(str(nums[k]),p.number_to_words(nums[k]))

    print(nstr)

1, 2 (3)
one, 2 (3)
1, two (3)
one, two (3)
1, 2 (three)
one, 2 (three)
1, two (three)
one, two (three)
-------------
1, 2 (3)
1, two (3)
1, two (3)
1, two (3)
1, two (3)
1, two (3)
1, two (3)
1, two (3)


In [156]:
[i for i in npbool(9) if i == True]

[True, True]

In [29]:
#search with substring
ret2 = url.urlopen(omdbSrchQuery("twelve")).read()
ret2d = ast.literal_eval(ret2) #convert to dict
#print((ret2d['totalResults'])) #how many results
t2d = pd.DataFrame(ret2d['Search']) #convert search results to DataFrame
print(t2d.keys()) #result columns

In [80]:
print (t2d['Title'])

0                 Twelve Monkeys
1                 Ocean's Twelve
2                         Twelve
3    The Twelve Tasks of Asterix
4            Twelve O'Clock High
5              The Twelve Chairs
6                  Twelve Chairs
7            The Twelve Kingdoms
8                  Twenty Twelve
9               Twelve Mile Road
Name: Title, dtype: object


In [100]:
imdbqstr = "http://www.imdb.com/xml/find?json=1&nr=1&tt=on&q=" #find?{json,xml,jsonp}
searchstr = "lost"
r3 = urllib2.urlopen(imdbqstr+searchstr).read().replace('\n','') #replace newline

In [125]:
r4 = re.sub(r"    <a href=(.*?)>",'',r3).replace('</a>...','') #replace links with nothing
print(r4[:400])

{"title_popular": [{ "id":"tt0411008", "title":"Lost", "name":"","title_description":"2004 TV series,     <a href='/name/nm0009190/'>J.J. Abrams</a>...","episode_title":"","description":"2004 TV series,     <a href='/name/nm0009190/'>J.J. Abrams</a>..."}],"title_exact": [{ "id":"tt0406942", "title":"Lost", "name":"","title_description":"2004/I,     <a href='/name/nm0501359/'>Darren Lemke</a>","epi

{"title_popular": [{ "id":"tt0411008", "title":"Lost", "name":"","title_description":"2004 TV series, J.J. Abrams","episode_title":"","description":"2004 TV series, J.J. Abrams"}],"title_exact": [{ "id":"tt0406942", "title":"Lost", "name":"","title_description":"2004/I, Darren Lemke</a>","episode_title":"","description":"2004/I, Darren Lemke</a>"},{ "id":"tt0391302", "title":"Lost", "name":"","tit


In [129]:
#test what happens if we query as fast as we possibly can
r30 = urllib2.urlopen("http://www.imdb.com/xml/find?json=1&nr=1&tt=on&q=lost").read().replace('\n','')

for i in range(100):
    r3 = urllib2.urlopen("http://www.imdb.com/xml/find?json=1&nr=1&tt=on&q=lost").read().replace('\n','')
    if(r30 != r3):
        print(r3)